In [2]:
import numpy as np
import qiskit
import qiskit_aer
import qiskit.visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime
mpl.rcParams.update(mpl.rcParamsDefault)

In [3]:
import qiskit_ibm_provider
ibmprovider = qiskit_ibm_provider.IBMProvider(token="9e027ffd514fe6fe384a7c915db815287e72104e52daca921f8d7854d6362b39a127bad928fb2ddad1d85bf7434306ca01b5496e7fab340eef01b815df13d45f", instance="ibm-q/open/main")
backend_sherbrooke = ibmprovider.get_backend('ibm_sherbrooke')
backend_kyoto = ibmprovider.get_backend('ibm_kyoto')

ModuleNotFoundError: No module named 'qiskit_ibm_provider'

In [13]:
from qiskit_ibm_runtime.fake_provider import FakePerth
fakebackend = FakePerth()

In [14]:
def circmake(probe,groups,points,backend):
    
    circs=[] 
    j = 0
    
    while j <groups:
        i = 0
        while i<points:
        
            circ = qiskit.QuantumCircuit(qiskit.QuantumRegister(1, name="Meter_Z"),
                      qiskit.QuantumRegister(1, name="System"),
                      qiskit.QuantumRegister(1, name="Probe_X"),
                      qiskit.QuantumRegister(1, name="Probe_Z"),
                      qiskit.ClassicalRegister(4, name="readout"))

            mez=np.arccos(0.0+1/(points-1)*i)

            circ.rz(-np.pi,1)
            circ.sx(1)
            circ.rz(-np.pi,1)
            circ.barrier()

            circ.rz(-np.pi,3)
            circ.sx(3)
            circ.rz(np.pi-np.arccos(probe),3)
            circ.sx(3)

            circ.cx(1,3)

            circ.barrier()
            circ.rz(np.pi/2,1)
            circ.sx(1)
            circ.rz(np.pi/2,1)

            circ.rz(-np.pi,2)
            circ.sx(2)
            circ.rz(np.pi-np.arccos(probe),2)
            circ.sx(2)

            circ.cx(1,2)

            circ.rz(np.pi/2,1)
            circ.sx(1)
            circ.rz(np.pi/2,1)

            circ.barrier()
            circ.rz(-np.pi,0)
            circ.sx(0)
            circ.rz(np.pi-mez,0)
            circ.sx(0)

            circ.cx(1,0)


            circ.barrier()
            circ.rz(np.pi/2,1)
            circ.sx(1)
            circ.rz(np.pi/2,1)
    
            circ.measure([0,1,2,3],[1,0,2,3])
            circ=qiskit.compiler.transpile(circ,backend=backend,optimization_level=1)
            circs.append(circ)
            i += 1
        j+= 1
    
    return circs

In [15]:
def make_noise_model(prob_1=None,prob_2=None,prob_r=None):
    cnoise_model = qiskit_aer.noise.NoiseModel(basis_gates=['cx', 'delay', 'id', 'measure', 'reset', 'rz', 'sx', 'x'])
    
    if prob_1:
        error_1 = qiskit_aer.noise.depolarizing_error(prob_1, 1)
        cnoise_model.add_all_qubit_quantum_error(error_1, ['rz', 'sx', 'x',"id"])
    
    if prob_2:
        error_2 = qiskit_aer.noise.depolarizing_error(prob_2, 2)
        cnoise_model.add_all_qubit_quantum_error(error_2, ['cx'])
    
    if prob_r:
        error_r = qiskit_aer.noise.ReadoutError(prob_r)
        cnoise_model.add_all_qubit_readout_error(error_r)
    
    return cnoise_model

In [16]:
def adjust_backend_noise_model(backend, phase=0, amplitude=0):
    qnoise_model = qiskit_aer.noise.NoiseModel.from_backend(backend,temperature=50)
    error_1 = qiskit_aer.noise.phase_amplitude_damping_error(phase, amplitude)
    
    m = backend.num_qubits
    for n in np.linspace(0, m - 1, m, dtype=int) :
        qnoise_model.add_quantum_error(error_1, ['rz', 'sx', 'x',"id"], [n])
    del m
    return qnoise_model

In [17]:
def get_ed(raw):
    err_raw = qiskit.result.marginal_counts(raw,indices=[1,3]).get_counts()
    dis_raw = qiskit.result.marginal_counts(raw,indices=[0,2]).get_counts()
    error = np.zeros([groups,points])
    i = 0
    while i < groups:
        j = 0
        while j < points:
            a = (np.sqrt(2*abs(1-(err_raw[i*points+j]['00']+err_raw[i*points+j]['11']-err_raw[i*points+j]['01']-err_raw[i*points+j]['10'])/shots/0.1)))
            error[i][j] = a
            j += 1
        i += 1
    
    disturb = np.zeros([groups,points])
    i = 0
    while i < groups:
        j = 0
        while j < points:
            a = (np.sqrt(2*abs(1-(dis_raw[i*points+j]['00']+dis_raw[i*points+j]['11']-dis_raw[i*points+j]['01']-dis_raw[i*points+j]['10'])/shots/0.1)))
            disturb[i][j] = a
            j += 1
        i += 1
    return error.transpose(), disturb.transpose()

In [18]:
def get_plot_data(raw_error,raw_disturb,points):
    result = np.zeros([4,points]) 
    i = 0
    while i <points :
        result[0,i]= (np.sum(raw_error[i])-np.max(raw_error[i])-np.min(raw_error[i]))/(groups-2)
        result[2,i]= (np.sum(raw_disturb[i])-np.max(raw_disturb[i])-np.min(raw_disturb[i]))/(groups-2)
        #result[0,i]= np.mean(raw_error[i])
        #result[2,i]= np.mean(raw_disturb[i])
        result[1,i]= np.std(raw_error[i],ddof=1)
        result[3,i]= np.std(raw_disturb[i],ddof=1)
        i += 1
    return result

In [19]:
def measurement_strength_range_plot(sim_result, points):
    E=np.linspace(0,1,1001)
    plt.figure(figsize=(15,10))
   
    strength1 = np.linspace(0.0,1.0,11)
    strength2 = np.linspace(0.0,1.0,points)

    plt.plot(E,np.sqrt(2*(1-E)),color='#0000de',lw=3,label='Theoretical error')
    plt.plot(E,np.sqrt(2*(1-np.sin(np.arccos(E)))),color='#de0000',lw=3,label='Theoretical disturbance')

#    plt.errorbar(strength1, exp_result[0], yerr=exp_result[1], elinewidth=1 ,color="b",fmt="^",ms=15,mec="b",capsize=5,capthick=2,label='Experiment error')
#    plt.errorbar(strength1, exp_result[2], yerr=exp_result[3], elinewidth=1 ,color="r",fmt="d",ms=17,mec="r",capsize=5,capthick=2,label='Experiment disturbance')

    plt.plot(strength2, sim_result[0], lw=3, ls="--" ,color="g",label='Simulated error')
    plt.plot(strength2, sim_result[2], lw=3, ls="--" ,color="purple",label='Simulated disturbance')

#    plt.errorbar(endata[0],endata[1], yerr=endata[3], elinewidth=1 ,color="g",fmt="^",ms=15,mec="g",capsize=5,capthick=2,label='Optical error')
#    plt.errorbar(endata[0],endata[2], yerr=endata[4], elinewidth=1 ,color="purple",fmt="d",ms=17,mec="purple",capsize=5,capthick=2,label='Optical disturbance')

    order = [2,0,3,1]
    handles, labels = plt.gca().get_legend_handles_labels()
    #add legend to plot
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],bbox_to_anchor=(0.87,-0.17),loc='upper right',fontsize=22,ncol=2) 
    plt.legend(bbox_to_anchor=(0.97,-0.17),loc='upper right',fontsize=22,ncol=2)
    plt.xlabel("Measurement Strength (cos ${\\theta}$)",size=32,labelpad=24)
    plt.ylabel("Error and Disturbance",size=32,labelpad=18)
    plt.tick_params(labelsize=32)
    plt.grid(True)
    plt.show()


In [20]:
def forbidden_zone_plot(sim_result):
    plt.figure(figsize=(15,15))

    b = np.linspace(0.5,1.8,1000)
    c = np.linspace(0.001,1,1000)


    plt.errorbar(sim_result[0],sim_result[2], xerr=sim_result[1],yerr=sim_result[3], elinewidth=2 ,color="black",capsize=5,capthick=2,fmt="o",ms=12,mec="b")

    x = np.linspace(-2, 2, 1000)

    x = np.linspace(-2, 2, 1000)
    y = np.linspace(-2, 2, 1000)
    X, Y = np.meshgrid(x, y)


    Z = np.sqrt((X**2 - X**4/4) + (Y**2 - Y**4/4))


    plt.contourf(X, Y, Z, levels=[1, Z.max()], colors='y', alpha=0.22)


    plt.contour(X, Y, Z, levels=[1], colors='blue',linewidths=4,linestyles="--")
    plt.plot(c,(1-c)/(1+c),lw=4,color="black")

    plt.plot(b,1/b,lw=4,color="purple")

    plt.plot(c,np.sqrt(1-c*c),lw=4,color="r")



    plt.fill_between(c,(1-c)/(1+c),color="gray",alpha=0.3)
    plt.fill_between(c,(1-c)/(1+c),np.sqrt(1-c*c),color="b",alpha=0.3)
    plt.fill_between(b,1/b,999,color="w",alpha=1)
    plt.fill_between(b,1/b,999,color="g",alpha=0.4)
    plt.xlim(0,1.6)
    plt.ylim(0,1.6)

    #plt.legend(bbox_to_anchor=(0.97,-0.12),loc='upper right',fontsize=22,ncol=2)
    plt.xlabel("Error",size=32)
    plt.ylabel("Disturbance",size=32)
    plt.tick_params(labelsize=32)
    plt.show()

In [21]:
endata = np.array([[1, 0.286356421, 1.393197761, 0.10687, 0.01384],
          [0.985, 0.360555128, 1.286468033, 0.07454, 0.01746],
          [0.94, 0.498998998, 1.170042734, 0.05173, 0.02122],
          [0.866, 0.68556546, 1.069579357, 0.03524, 0.02645],
          [0.766, 0.80684571, 0.908845421, 0.02983, 0.02114],
          [0.643, 0.927900857, 0.797496081, 0.0284, 0.02203],
          [0.5, 1.105893304, 0.628490254, 0.03243, 0.04083],
          [0.342, 1.221474519, 0.559464029, 0.01564, 0.05626],
          [0.174, 1.354252561, 0.46151923, 0.0143, 0.05765],
          [0, 1.439791652, 0.354964787, 0.02433, 0.06816]])
endata = endata.transpose()
exp_result = np.array([[1.41552695,1.36435836,1.28781354,1.20447702,1.16895506,1.06177519,
                      1.01689894,0.88108358,0.74446709,0.68535111,0.56322395],
                        [0.01512828,0.03275893,0.03075678,0.02201124,0.0278425, 0.05022082,
                      0.05120571,0.06349935,0.07895512,0.10610931,0.13539114],
                        [0.65089794,0.58770411,0.61701889,0.70274453,0.71313554,0.78121456,
                      0.86118775,0.93261966,1.02441124,1.09042051,1.34376592],
                        [0.08422546,0.08669819,0.08041575,0.09286758,0.07725798,0.06533498,
                      0.06205029,0.0433318, 0.05103344,0.05436436,0.04136799]])

In [36]:
shots = 10000
probe_strength = 0.1
groups = 10
backend = fakebackend
points = 11

circs = circmake(probe_strength,groups,points,backend)

qsimbackend = qiskit_aer.AerSimulator(method='density_matrix')

noise_model_list = [qiskit_aer.noise.NoiseModel.from_backend(fakebackend,temperature=50)]

qnoise_model = qiskit_aer.noise.NoiseModel.from_backend(fakebackend,temperature=50)
error_1 = qiskit_aer.noise.phase_amplitude_damping_error(0.01, 0.0001)
for n in range(0,7):
    qnoise_model.add_quantum_error(error_1, ['sx', 'x',"id"], [n])


sim_job = fakebackend.run(circs, noise_model = qnoise_model ,shots=shots).result()

In [35]:
qnoise_model = qiskit_aer.noise.NoiseModel.from_backend(fakebackend,temperature=50)
error_1 = qiskit_aer.noise.phase_amplitude_damping_error(0.01, 0.01)
qnoise_model.add_quantum_error(error_1, ['sx', 'x',"id"], [1])
sim_job = fakebackend.run(circs, noise_model = qnoise_model ,shots=shots).result()

In [25]:
shots = 10000
probe_strength = 0.1
groups = 10
backend = fakebackend
points = 11

circs = circmake(probe_strength,groups,points,backend)
sim_job = fakebackend.run(circs, noise_model = qnoise_model ,shots=shots).result()

In [ ]:
circs = circmake(probe_strength,groups)
sim_job= csimbackend.run(circs,shots=shots).result()
raw_error,raw_disturb = get_ed(sim_job)
sim_result = get_plot_data(raw_error,raw_disturb)

In [ ]:
measurement_strength_plot(sim_result)

In [ ]:
forbidden_zone_plot(sim_result)